In [ ]:
'''정리노트 #1'''

In [3]:
import numpy as np

'''수동 역전파'''

class Variable:
    def __init__(self,data):
        self.data = data
        self.grad= None

In [4]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input # 입력 변수를 기억(보관)함 / 역전파 시 필요함
        return output

    def forward(self, x):
        raise NotImplementedError() # 순전파
    
    def backward(self, gy):
        raise NotImplementedError() # 역전파 

In [5]:
class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx # 인수로 전달된 미분에 'y = x ** 2'의 미분을 곱한 값이 backward의 결과

In [6]:
class Exp(Function): 
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [7]:
#순전파 계산 코드

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [8]:
#역전파로 y 미분하기 (C -> B -> A 순서)

y.grad = np.array(1.0) # 출력 y의 미분값을 np.array(1.0)로 설정
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [9]:
'''역전파 자동화 : 일반적인 계산(순전파)을 해 주면 자동으로 역전파가 이루어지는 구조'''

class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func): #인스턴스 변수 creator추가
        self.creator = func

In [10]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self) # 출력 변수에 창조자 설정
        self.input = input
        self.output = output # 출력도 저장하기
        return output

In [11]:
class Square(Function):
    def forward(self,x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx # 인수로 전달된 미분에 'y = x ** 2'의 미분을 곱한 값이 backward의 결과

In [12]:
class Exp(Function): 
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [13]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 계산 그래프의 노드들을 거꾸로 거슬러 올라감
assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

#assert 함수는 그 결과가 True 가 아니면 예외 발생 / 문제 없이 실행되므로 조건을 모두 충족

In [14]:
'''변수 y에서 b까지의 역전파'''
y.grad = np.array(1.0)
C = y.creator # 함수를 가져옴
b = C.input # 함수의 입력을 가져옴
b.grad = C.backward(y.grad) # 함수의 backward 메서드를 호출함.

'''변수 b에서 a까지의 역전파'''
B = b.creator 
a = B.input
a.grad = B.backward(b.grad)

'''변수 a에서 x로의 역전파'''
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [15]:
'''반복 작업을 자동화하기 위해 variable 클래스에 backward라는 새 메서드 추가'''

class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        f = self.creator # 함수를 가져옴
        if f is not None:
            x = f.input # 함수의 입력을 가져옴
            x.grad = f.backward(self.grad) # 함수의 backward 메서드를 호출
            x.backward() # 하나 앞 변수의 backward 메서드를 호출한다(재귀)

In [16]:
# 자동 실행
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

#역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [17]:
'''반복문을 이용해 구현하기(재귀 -> 반복문)'''
'''처리해야 할 함수들을 funcs라는 리스트에 차례대로 집어넣는다.'''

class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        funcs = [self.creator] 
        while funcs:
            f = funcs.pop() # 함수를 가져옴
            x, y = f.input, f.output # 함수의 입력과 출력을 저장
            x.grad = f.backward(y.grad) # backward 메서드를 호출
            
            if x.creator is not None:
                funcs.append(x.creator) # 하나 앞의 함수를 리스트에 추가

In [18]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

#역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [ ]:
# 재귀는 함수를 재귀적으로 호출 할 때마다 중간 결과를 메모리에 유지하면서 처리, 따라서 일반적으로 반복문이 효율이 더 좋다.

In [19]:
'''파이썬 함수로 이용하기'''
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

In [20]:
x = Variable(np.array(0.5))
y = square(exp(square(x))) #연속하여 적용!
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


In [22]:
'''backward 메서드 간소화하기'''
class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) #np.ones_like(self.data)는 self.data와 형상과 타입이 같은 ndarray인스턴스 생성.
                                                #모든 요소를 1로 채워서 돌려줌
            
        funcs = [self.creator] 
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output 
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)
                
x = Variable(np.array(0.5))
y = square(exp(square(x))) 
y.backward()
print(x.grad)

3.297442541400256


In [2]:
'''테스트 하기'''
import unittest

class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expect = np.array(4.0)
        self.assertEqual(y.data, expected) # 주어진 두 객체가 동일한지 여부를 판정